In [2]:
def chatbot():
    import pandas as pd
    import numpy as np
    from textblob import TextBlob
    from sklearn.preprocessing import LabelEncoder
    import string
    from sklearn.feature_extraction.text import CountVectorizer
    
    df = pd.read_json('intents.json')
    # intents' is the series in your DataFrame make it all columns 
    df = pd.DataFrame(df['intents'].tolist())
    
    # label encoded the tag column
    le = LabelEncoder()
    df['tag'] = le.fit_transform(df['tag'])
    
    # create two different df for convinience
    response_df = pd.DataFrame({'tag': df['tag'], 'response': df['responses']})
    pattern_df = pd.DataFrame({'tag': df['tag'], 'patterns': df['patterns']})
    
    # Use explode to split the lists into separate rows
    pattern_df = pattern_df.explode('patterns')
    response_df = response_df.explode('response')
    # Reset index if needed
    pattern_df = pattern_df.reset_index(drop=True)
    response_df = response_df.reset_index(drop=True)
    
    # Shuffle or jumble the rows
    pattern_df = pattern_df.sample(frac=1, random_state=np.random.seed(42))
    
    # Data preprocessing
    pattern_df['patterns'] = pattern_df['patterns'].str.lower()
    pattern_df['patterns'] = pattern_df['patterns'].str.replace('[{}]'.format(string.punctuation), '')
    import re
    pattern_df['patterns'] = pattern_df['patterns'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
    # The above line of code removes all punctuation from the column
    
    # Traing the dataset 
    patterns = list(pattern_df['patterns'])
    cv = CountVectorizer()
    patt = cv.fit_transform(patterns).toarray()
    # creating a temporary dataframe to store the bow of pattern and then add the tag column to it for training
    temp_df = pd.DataFrame(patt, index= pattern_df.index)
    temp_df ['tag']= pattern_df['tag']
    
    # training the chatbot for entire data 
    X_train = temp_df.iloc[: , :-1].values
    y_train = temp_df.iloc[:, -1].values
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    rf = RandomForestClassifier()
    rf.fit(X_train,y_train)

    def remove_pun(input_text):
        input_text = re.sub('[%s]' % re.escape(string.punctuation), '', input_text)
        input_text = input_text.lower()
        return input_text
    response_df_sampled = response_df.sample(frac=1, random_state=np.random.seed(42)) 
    def get_response(predicted_tag, p):
        # Check if the predicted tag exists in the DataFrame
        if p > 0.4:
            possible_responses = response_df_sampled[response_df_sampled['tag'] == predicted_tag]['response']
            
            # Choose a random response from the list of possible responses
            selected_response = np.random.choice(possible_responses)
            # probabilities = rf.predict_proba(X_test)
            
            # print(f'Probability: {probabilities}')
            print(f'ChatBot: {selected_response}')
        else:
            print(f'Sorry I don\'t understand this !!!')
            possible_responses = ['Sorry I don\'t understand this !!!',
                                 'Can you further ellobrate',
                                 'Be more precise!',
                                 'I don\'t get it !']
            selected_response = np.random.choice(possible_responses)
            
################ the actual main function begins    
    print("Hello! I'm your Chatbot. Type 'exit' to end the conversation.")
    while True:
        # Get user input
        user_input = input("You: ")
        textblb = TextBlob(user_input)
        user_input=textblb.correct().string
        # Check for exit command
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye! Have a great day.")
            break
        
        # Preprocess user input
        user_input = remove_pun(user_input)
        
        # Transform user input into bag-of-words
        user_input_bow = cv.transform([user_input])
        
        # Predict the tag for the user input
        predicted_tag = rf.predict(user_input_bow)[0]
        
        # Predict probabilities for the test set
        global probabilities 
        probabilities = rf.predict_proba(user_input_bow)
        
        your_tag_index = list(rf.classes_).index(predicted_tag)
        
        # Access probabilities for the specified tag
        tag_probabilities = probabilities[:, your_tag_index]
        get_response(predicted_tag,tag_probabilities)
        # print(f'Prabability: {tag_probabilities}')

# Call the chatbot function to start the interaction
chatbot()

Hello! I'm your Chatbot. Type 'exit' to end the conversation.


You:  hello


ChatBot: Hello! How can I help you.


You:  college location


ChatBot: Jatni, Bhubaneswar Odisha


You:  exit


Chatbot: Goodbye! Have a great day.
